In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

from pkg import u,m

clen_values, photon_energy_values = [1.5, 2.5, 3.5], [6000, 7000, 8000]
param_matrix = u.parameter_matrix(clen_values, photon_energy_values)
print(param_matrix, '\n')

dataset_dict = {
    '01': [clen_values[0], photon_energy_values[0]],
    '02': [clen_values[0], photon_energy_values[1]],
    '03': [clen_values[0], photon_energy_values[2]],
    '04': [clen_values[1], photon_energy_values[0]],
    '05': [clen_values[1], photon_energy_values[1]],
    '06': [clen_values[1], photon_energy_values[2]],
    '07': [clen_values[2], photon_energy_values[0]],
    '08': [clen_values[2], photon_energy_values[1]],
    '09': [clen_values[2], photon_energy_values[2]],
}

# parameters
dataset = '01'
print(dataset_dict[dataset])
clen, photon_energy = dataset_dict[dataset]

threshold = 1

# instances
pm = u.PathManager()
peak_paths, water_peak_paths, labels, water_background_path = pm.select_dataset(dataset=dataset)
p = u.Processor(paths=pm, dataset=dataset)
dm = u.DatasetManager(paths=pm, dataset=dataset, parameters=(clen, photon_energy), transform=None, include_water_background=True, percent_water_repeat=0.35)

# peak, label, overlay, background are valid types
u.check_attributes(paths=pm, dataset=dataset, type='peak') 
u.check_attributes(paths=pm, dataset=dataset, type='overlay')
u.check_attributes(paths=pm, dataset=dataset, type='label')

[[(1.5, 6000.) (1.5, 7000.) (1.5, 8000.)]
 [(2.5, 6000.) (2.5, 7000.) (2.5, 8000.)]
 [(3.5, 6000.) (3.5, 7000.) (3.5, 8000.)]] 

[1.5, 6000]
Found water background image: water01.h5
Attributes 'clen' and 'photon_energy' assigned to /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/water/01/water01.h5
Attributes 'clen' and 'photon_energy' assigned to /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/labels/01/empty_water01.h5

Total images (true peak count): 39
Total images (including water background): 52
Water background images added: 13

Number of peak images: 52
Number of water images: 39
Number of label images: 52
Check: Path to water background image: /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/water/01/water01.h5

All files in dataset 01 of type 'peak' have matching attributes.
All files in dataset 01 of type 'overlay' have matching attributes.
All files in dataset 01 of type 'label' have matching attributes.


ValueError: Invalid type specified.

In [ ]:
# import subprocess

# runs process_directory.py, 
#   checks uniformity of images/
#   generates all, with corresponding water/*/.h5
#       labels/ 
#       peaks_water_overlay/ 
%run process_directory.py ../../images 

# subprocess.run(["python", "process_directory.py", "../../images"])
# p.process_directory(dataset=dataset, clen=clen, photon_energy=photon_energy)


Directories '01' through '09' already created in '../../images/labels'.
Directories '01' through '09' already created in '../../images/peaks'.
Directories '01' through '09' already created in '../../images/peaks_water_overlay'.
Directories '01' through '09' already created in '../../images/water'.
All directories in '../../images/peaks/01', '../../images/labels/01', and '../../images/peaks_water_overlay/01' have 39 images, as expected.
All directories in '../../images/peaks/02', '../../images/labels/02', and '../../images/peaks_water_overlay/02' have 17 images, as expected.
All directories in '../../images/peaks/03', '../../images/labels/03', and '../../images/peaks_water_overlay/03' have 15 images, as expected.
All directories in '../../images/peaks/04', '../../images/labels/04', and '../../images/peaks_water_overlay/04' have 14 images, as expected.
All directories in '../../images/peaks/05', '../../images/labels/05', and '../../images/peaks_water_overlay/05' have 15 images, as expect

KeyboardInterrupt: 

In [ ]:
# generate peak/overlay/label and upate attributes
# overlaying the correct water01.h5 path to the peak images
# p.process_directory(dataset=dataset, clen=clen, photon_energy=photon_energy)

# peak, label, overlay are valid types
u.check_attributes(paths=pm, dataset=dataset, type='peak') 
u.check_attributes(paths=pm, dataset=dataset, type='overlay')
u.check_attributes(paths=pm, dataset=dataset, type='label')


train_loader, test_loader = u.prepare(data_manager=dm, batch_size=10)


true parameter matrix...

All files in dataset 01 of type 'label' have matching attributes.

Data prepared.
Train size: 31
Test size: 8
Batch size: 10
Number of batches in train_loader: 4 



In [ ]:
model1 = m.BasicCNN1()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cfg = {
    'num_epochs': 10,
    'num_classes': 2,
    'batch_size': train_loader.batch_size,
    'test_size': len(train_loader.dataset),
    'test_size': len(test_loader.dataset),
    'criterion': criterion,
    'optimizer': optimizer,
    'device': device,
    'model': model1,
}

# arguments: self, model, loader: list, criterion, optimizer, device, cfg: dict
t = u.TrainTestModels(model=model1, loader=[train_loader, test_loader], criterion=criterion, optimizer=optimizer, device=device, cfg=cfg)
t.train()

Model training: BasicCNN1
-- epoch 0
Train loss: 0.2645605504512787
Train accuracy: 0.9999687671661377
-- epoch 1
Train loss: 0.2571524798870087
Train accuracy: 0.9999950528144836
-- epoch 2
Train loss: 0.24958244562149048
Train accuracy: 0.9999980330467224
-- epoch 3
Train loss: 0.24180715680122375
Train accuracy: 0.9999991059303284
-- epoch 4
Train loss: 0.23375419974327089
Train accuracy: 0.9999992847442627
-- epoch 5
Train loss: 0.2254271149635315
Train accuracy: 0.9999992847442627
-- epoch 6
Train loss: 0.21685744524002076
Train accuracy: 0.9999992847442627
-- epoch 7
Train loss: 0.2079702615737915
Train accuracy: 0.9999992847442627
-- epoch 8
Train loss: 0.1987803488969803
Train accuracy: 0.9999992847442627
-- epoch 9
Train loss: 0.1893494665622711
Train accuracy: 0.9999992847442627
